Important info has (*)
- Block image transfer **blit** one surface on top of another surface
- rotozoom (surface,angle,scale)

**sprite class**
- A class that contains a surface and a rectangle
- create a class for:
    - player
    - obstacle
- place sprites in group or groupsingle
- draw/update sprites into group (functions)

**groups in sprites** (important for collisions)
- group: a group for multiple sprites
- GroupSingle: a group with a single sprite

In [5]:
import pygame as pg
from sys import exit #this helps to close code 
from random import randint, choice #pick random item from list

#inherit
class Player(pg.sprite.Sprite):
    def __init__(self):
        super().__init__()
        player_walk_1 = pg.image.load('graphics/player/sakura1.png').convert_alpha()
        player_walk_1 = pg.transform.rotozoom(player_walk_1,0,0.8)
        player_walk_2 = pg.image.load('graphics/player/sakura2.png').convert_alpha()
        player_walk_2 = pg.transform.rotozoom(player_walk_2,0,0.8)
        self.player_walk = [player_walk_1,player_walk_2]
        self.player_index = 0 # to pick player 1,2
        player_jump = pg.image.load('graphics/player/sakura_jump.png')
        player_jump = pg.transform.rotozoom(player_jump,0,0.8)
        self.player_jump = player_jump
        self.image = self.player_walk[self.player_index]
        self.rect = self.image.get_rect(midbottom = (80,300))
        self.gravity = 0
        
        self.jump_sound = pg.mixer.Sound('audio/jump_magic.wav')
        self.jump_sound.set_volume(0.5)
    
    def player_input(self):
        keys = pg.key.get_pressed()
        if keys[pg.K_SPACE] and self.rect.bottom >= 300:
            self.gravity = -20
            self.jump_sound.play()
    
    def apply_gravity(self):
        self.gravity += 1
        self.rect.y += self.gravity
        if self.rect.bottom >= 300:
            self.rect.bottom = 300
            
    def animation_state(self):
        if self.rect.bottom < 300:
            self.image = self.player_jump
        else:
            self.player_index += 0.1
            if self.player_index >= len(self.player_walk):self.player_index = 0
            self.image = self.player_walk[int(self.player_index)]
    
    def update(self):
        self.player_input()
        self.apply_gravity()
        self.animation_state()
        
        
class Obstacle(pg.sprite.Sprite):
    def __init__(self,type):
        super().__init__()
        
        if type == 'fly':
            fly_1 = pg.image.load('graphics/fly/kero1.png').convert_alpha()
            fly_1 = pg.transform.rotozoom(fly_1,0,0.4)
            fly_2 = pg.image.load('graphics/fly/kero2.png').convert_alpha()
            fly_2 = pg.transform.rotozoom(fly_2,0,0.4)
            self.frames = [fly_1,fly_2]
            y_pos = 150
        
        else:
            obs_1 = pg.image.load('graphics/obstacle/penguin1.png').convert_alpha()
            obs_1 = pg.transform.rotozoom(obs_1,0,0.3)
            obs_2 = pg.image.load('graphics/obstacle/penguin2.png').convert_alpha()
            obs_2 = pg.transform.rotozoom(obs_2,0,0.3)
            self.frames = [obs_1,obs_2]
            y_pos = 300
        
        self.animation_index = 0
        self.image = self.frames[self.animation_index]
        self.rect = self.image.get_rect(midbottom = (randint(900,1100),y_pos))
        
    
    def animation_state(self):
        self.animation_index += 0.1
        if self.animation_index >= len(self.frames): self.animation_index = 0
        self.image = self.frames[int(self.animation_index)]

        
    def update(self):
        self.animation_state()
        self.rect.x -= 6
        self.destroy()
        
    def destroy(self):
        if self.rect.x <= -100:
            self.kill()
        
        
def display_score():
    current_time = int(pg.time.get_ticks() / 1000) - start_time  # Gives time in miliseconds
    score_surf = test_font.render(f'Score: {current_time}', False, (248, 255, 156))
    score_rect = score_surf.get_rect(center = (400,50))
    screen.blit(score_surf, score_rect)
    return current_time

    
def collision_sprite():
    #statement returns a list; #if sprite collides with group if it will be destroyed (boolean)
    if pg.sprite.spritecollide(player.sprite,obstacle_group,False):
        #this helps for collision and not start in the same spot causing eternal crash (gamevoverscreen)
        obstacle_group.empty()
        return False
    else:
        return True 
    
    
    
#Initialize (*)
pg.init() 
# Display surface
screen = pg.display.set_mode((800,400)) #width, height
# Starting variables [2]
pg.display.set_caption('sakura')
icon = pg.image.load('graphics/sakuralogo.png')
pg.display.set_icon(icon)
clock = pg.time.Clock()
test_font = pg.font.Font('font/Pixeltype.ttf', 50) #font type, font size
game_active = False
start_time = 0
score = 0
background_music = pg.mixer.Sound('audio/sakura_instrumental.wav')
background_music.set_volume(0.3)
background_music.play(loops = -1)

# Groups: a group that contains a sprite
player = pg.sprite.GroupSingle()
player.add(Player())

obstacle_group = pg.sprite.Group()


# SURFACES
sky_surface = pg.image.load('graphics/sky2.png').convert()
sky_surface = pg.transform.scale(sky_surface,(800,300))
ground_surface = pg.image.load('graphics/ground3.png').convert()
#score_surf = test_font.render('My sakura game', False,(64,64,64) )# Text, Anti Alising(False = best for pixel art, color
#score_rect = score_surf.get_rect(center = (400,50))



#Intro screen
player_stand = pg.image.load('graphics/sakura_screen.png').convert_alpha()
player_stand_rect = player_stand.get_rect(center = (400,200))

game_title = test_font.render(' Sakura Runner ',False,(64,64,64))
game_title_rect = game_title.get_rect(center = (400,90))

instructions = test_font.render('  Press space to run  ', False, (64,64,64))
inst_rect = instructions.get_rect(center = (400,330))

# Timer
obstacle_timer = pg.USEREVENT + 1 #pygame already has events; avoid conflict = +1 for each customed event
pg.time.set_timer(obstacle_timer,1400)#event, miliseconds



# RUN GAME [3]
while True:
    
    #EVENT LOOP; All possible player's input we use a for loop, Gets all events
    for event in pg.event.get():
        if event.type == pg.QUIT:
            pg.quit()
            exit()
        
        
        if game_active:
            if event.type == obstacle_timer:
                obstacle_group.add(Obstacle(choice(['fly','obs','obs','obs'])))
              
        else:
            if event.type == pg.KEYDOWN and event.key == pg.K_SPACE:
                game_active = True
                start_time = int(pg.time.get_ticks() / 1000)    
              
        
    
    if game_active:
        # Static surfaces
        screen.blit(sky_surface, (0,0)) #surface, position
        screen.blit(ground_surface,(0,300))
        score = display_score()


        # Player
        player.draw(screen)
        player.update()
        
        obstacle_group.draw(screen)
        obstacle_group.update()
        
        
        # Obstacle Movement 
        #obstacle_rect_list = obstacle_movement(obstacle_rect_list) #update de list (obstacle moved slightly to left)
        
       # Collisions
        game_active = collision_sprite()
        #game_active = collisions(player_rect,obstacle_rect_list)
        
    else:
        # Game over / intro screen
        screen.fill((247, 203, 235)) # Light yellow 248, 255, 156, purple 156, 156, 255, pink 247, 203, 235
        screen.blit(player_stand, player_stand_rect)
        
        score_msg = test_font.render(f'Your score: {score}', False, (64,64,64))
        score_msg_rect = score_msg.get_rect(center = (400,330))
        
        #pg.draw.line(screen,(227, 25, 126),(150,330),(650,330),5)
        #pg.draw.line(screen,(227, 25, 126),(150,90),(650,90),5)
        #pg.draw.line(screen,(227, 25, 126),(150,90),(150,330),5)
        #pg.draw.line(screen,(227, 25, 126),(650,90),(650,330),5)
        
        #pg.draw.rect(screen,(248, 255, 156),inst_rect)
        pg.draw.rect(screen,(227, 25, 126),inst_rect,3,8,-3,-3,-5,-5)
        #pg.draw.rect(screen,(248, 255, 156),game_title_rect)
        pg.draw.rect(screen,(227, 25, 126),game_title_rect,3,8,-3,-3,-5,-5)
        screen.blit(game_title, game_title_rect)
        
        if score == 0:screen.blit(instructions, inst_rect)
        else:screen.blit(score_msg, score_msg_rect)
        
    
    # Updates the display surface(*)
    pg.display.update() 
    #calling frame rate (speed) 60fps; keep in mind for more sophisticated games
    clock.tick(60) 

SystemExit: 